![picture1.png](picture1.png)

In [1]:
from keras import backend as K

def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [3]:
# imports for array-handling and plotting
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# keras imports for the dataset and building our neural network
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Conv2D, Flatten, MaxPooling2D
from keras.utils import np_utils, to_categorical
from keras.wrappers.scikit_learn import KerasClassifier


# sklearn imports for the model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

import time

#Data-preparation

images = np.loadtxt("handwritten_digits_images.csv", delimiter=',')
labels = np.loadtxt("handwritten_digits_labels.csv", delimiter=',')

X_train, X_test, y_train, y_test = train_test_split(images, labels,test_size=0.3, random_state=32)

#reshape data to fit model
X_train = X_train.reshape(49000,28,28,1)
X_test = X_test.reshape(21000,28,28,1)

#Normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalizing the RGB codes by dividing it to the max RGB value.
X_train /= 255
X_test /= 255


def make_model():
    
    model = Sequential()

    model.add(Conv2D(10,kernel_size=5, activation='relu', input_shape=(28,28,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(20, kernel_size=5, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy',    metrics=['accuracy', f1, precision, recall])   
    
    return model


def tuning(X_train,Y_train,X_test,Y_test):

    
    epochs = list(range(1,5))
    param_grid = dict(nb_epoch=epochs)

    k_model = KerasClassifier(build_fn=make_model, verbose=0)
   
    clf = GridSearchCV(estimator=k_model, param_grid=param_grid, 
                                   cv=5,
                                   scoring="accuracy", verbose=0 ,n_jobs=-1)
    clf.fit(X_train,Y_train)
    
    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    
    model=make_model()
    
    return clf,model

tune_start = time.time()
clf = tuning(X_train,y_train,X_test,y_test)
tune_end = time.time()

tuning_time=tune_end-tune_start

Best parameters set found on development set:

{'nb_epoch': 3}


In [4]:
model=clf[1]

y_train_enc = to_categorical(y_train)
y_test_enc = to_categorical(y_test)

start = time.time()
history = model.fit(X_train, y_train_enc, validation_data=(X_test, y_test_enc), epochs=3)
end = time.time()

exec_time = end - start

evaluation = model.evaluate(X_test, y_test_enc, verbose=2)

Train on 49000 samples, validate on 21000 samples
Epoch 1/3
49000/49000 [==============================] - 13s 269us/step - loss: 0.2717 - acc: 0.9170 - f1: 0.9034 - precision: 0.9300 - recall: 0.8869 - val_loss: 0.0949 - val_acc: 0.9718 - val_f1: 0.9720 - val_precision: 0.9769 - val_recall: 0.9673
Epoch 2/3
49000/49000 [==============================] - 14s 282us/step - loss: 0.0978 - acc: 0.9697 - f1: 0.9702 - precision: 0.9749 - recall: 0.9657 - val_loss: 0.0619 - val_acc: 0.9825 - val_f1: 0.9826 - val_precision: 0.9855 - val_recall: 0.9800
Epoch 3/3
49000/49000 [==============================] - 15s 310us/step - loss: 0.0719 - acc: 0.9780 - f1: 0.9783 - precision: 0.9814 - recall: 0.9753 - val_loss: 0.0521 - val_acc: 0.9845 - val_f1: 0.9843 - val_precision: 0.9866 - val_recall: 0.9821


In [5]:
loss = evaluation[0]
accuracy = evaluation[1]
f1_measure = evaluation[2]
precision = evaluation[3]
recall = evaluation[4]

In [11]:
print()
print("Test Loss : ", loss)
print("Test Accuracy : ", round(accuracy*100, 2), "%")
print("F1-Measure", round(f1_measure*100, 4))
print("Precision", round(precision*100, 4))
print("Recall", round(recall*100, 4))
print("Cross Validation Time : ", round(tuning_time, 2), "sec" )
print("Training Time : ",  round(exec_time, 2), "sec"  )


predicted_classes = model.predict_classes(X_test)
cm = confusion_matrix(y_test, predicted_classes)

print()
print('Confusion Matrix : ')
print()
print(cm)
print()

# see which we predicted correctly and which not ----- Indices of elements that are non-zero.
correct_indices = np.nonzero(predicted_classes == y_test)[0]
incorrect_indices = np.nonzero(predicted_classes != y_test)[0]

print()
print(len(correct_indices)," classified correctly")
print(len(incorrect_indices)," classified incorrectly")

error_rate = len(incorrect_indices)/(len(correct_indices)+len(incorrect_indices))

print()
print("Error rate : ", round(error_rate*100, 2), "%")


Test Loss :  0.052068482326787145
Test Accuracy :  98.45 %
F1-Measure 98.4341
Precision 98.6622
Recall 98.2143
Cross Validation Time :  225.95 sec
Training Time :  42.43 sec

Confusion Matrix : 

[[2012    0    6    1    1    2    8    0    2    6]
 [   1 2359   14    0    2    1    3    4    3    1]
 [   0    1 2042    6    0    0    0    8    3    1]
 [   1    0   18 2129    0    3    0    3    3    5]
 [   2    2    3    0 2027    0    5    5    1   11]
 [   1    1    5   14    1 1839   12    2    4    1]
 [   4    1    0    0    2    2 2063    0    2    0]
 [   0    5   22    2    3    0    0 2128    2    3]
 [   4    9    7    6    6    9    3    5 1989   12]
 [   3    4    0    6    4    8    1   10    4 2086]]


20674  classified correctly
326  classified incorrectly

Error rate :  1.55 %
